In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Charger le dataset depuis un fichier CSV
df = pd.read_csv('/content/imdb_top_1000.csv')

# Afficher les premières lignes pour inspecter les données
print(df.head())


                                         Poster_Link  \
0  https://m.media-amazon.com/images/M/MV5BMDFkYT...   
1  https://m.media-amazon.com/images/M/MV5BM2MyNj...   
2  https://m.media-amazon.com/images/M/MV5BMTMxNT...   
3  https://m.media-amazon.com/images/M/MV5BMWMwMG...   
4  https://m.media-amazon.com/images/M/MV5BMWU4N2...   

               Series_Title Released_Year Certificate  Runtime  \
0  The Shawshank Redemption          1994           A  142 min   
1             The Godfather          1972           A  175 min   
2           The Dark Knight          2008          UA  152 min   
3    The Godfather: Part II          1974           A  202 min   
4              12 Angry Men          1957           U   96 min   

                  Genre  IMDB_Rating  \
0                 Drama          9.3   
1          Crime, Drama          9.2   
2  Action, Crime, Drama          9.0   
3          Crime, Drama          9.0   
4          Crime, Drama          9.0   

                         

In [ ]:
# Vérifier les valeurs manquantes
print(df.isnull().sum())



Poster_Link        0
Series_Title       0
Released_Year      0
Certificate      101
Runtime            0
Genre              0
IMDB_Rating        0
Overview           0
Meta_score       157
Director           0
Star1              0
Star2              0
Star3              0
Star4              0
No_of_Votes        0
Gross            169
dtype: int64


In [ ]:
print(df.columns)


Index(['Poster_Link', 'Series_Title', 'Released_Year', 'Certificate',
       'Runtime', 'Genre', 'IMDB_Rating', 'Overview', 'Meta_score', 'Director',
       'Star1', 'Star2', 'Star3', 'Star4', 'No_of_Votes', 'Gross'],
      dtype='object')


In [ ]:
# Supprimer les lignes avec des valeurs manquantes dans les colonnes 'Overview' et 'Series_Title'
df = df.dropna(subset=['Overview', 'Series_Title'])


In [ ]:
import re

# Fonction de nettoyage des résumés
def clean_text(text):
    text = text.lower()  # Mettre en minuscules
    text = re.sub(r'http\S+', '', text)  # Supprimer les URL
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Supprimer les caractères non alphabétiques
    text = re.sub(r'\s+', ' ', text)  # Remplacer les espaces multiples par un seul
    return text

# Appliquer la fonction de nettoyage sur la colonne 'Overview'
df['Overview'] = df['Overview'].apply(clean_text)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialiser le TF-IDF Vectorizer
tfidf = TfidfVectorizer(max_features=5000, stop_words='english')

# Appliquer le vectorizer sur la colonne 'Overview' pour créer une matrice de caractéristiques
X = tfidf.fit_transform(df['Overview'])

# Afficher la forme de la matrice TF-IDF
print(X.shape)


(1000, 5000)


In [ ]:
# Supprimer les colonnes non nécessaires
df = df.drop(columns=['Poster_Link', 'Meta_score', 'Star1', 'Star2', 'Star3', 'Star4'])


In [ ]:
# Vérifier les premières lignes pour s'assurer que le nettoyage est effectué correctement
print(df.head())


df.to_csv('imdb_movies_cleaned.csv', index=False)


               Series_Title Released_Year Certificate  Runtime  \
0  The Shawshank Redemption          1994           A  142 min   
1             The Godfather          1972           A  175 min   
2           The Dark Knight          2008          UA  152 min   
3    The Godfather: Part II          1974           A  202 min   
4              12 Angry Men          1957           U   96 min   

                  Genre  IMDB_Rating  \
0                 Drama          9.3   
1          Crime, Drama          9.2   
2  Action, Crime, Drama          9.0   
3          Crime, Drama          9.0   
4          Crime, Drama          9.0   

                                            Overview              Director  \
0  two imprisoned men bond over a number of years...        Frank Darabont   
1  an organized crime dynastys aging patriarch tr...  Francis Ford Coppola   
2  when the menace known as the joker wreaks havo...     Christopher Nolan   
3  the early life and career of vito corleone in .

In [ ]:
import re

def clean_text(text):
    # Convertir en minuscules
    text = text.lower()
    # Supprimer les URL
    text = re.sub(r'http\S+', '', text)
    # Supprimer les caractères non alphabétiques (sauf les espaces)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Remplacer les espaces multiples par un seul
    text = re.sub(r'\s+', ' ', text)
    return text

# Appliquer le nettoyage sur la colonne 'Overview'
df['Overview'] = df['Overview'].apply(clean_text)


In [ ]:
import nltk
from nltk.corpus import stopwords

# Télécharger les stop words de NLTK
nltk.download('stopwords')

# Créer une fonction pour supprimer les stopwords
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    tokens = text.split()
    tokens_cleaned = [word for word in tokens if word not in stop_words]
    return " ".join(tokens_cleaned)

# Appliquer la suppression des stopwords sur la colonne 'Overview'
df['Overview'] = df['Overview'].apply(remove_stopwords)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialisation du TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')

# Appliquer le TF-IDF pour transformer les résumés en vecteurs
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Overview'])

# Afficher la forme de la matrice TF-IDF pour vérifier
print(tfidf_matrix.shape)


(1000, 5000)


In [ ]:
# Convertir les évaluations IMDb en nombres flottants
df['IMDB_Rating'] = pd.to_numeric(df['IMDB_Rating'], errors='coerce')

# Supprimer les films avec des évaluations manquantes
df = df.dropna(subset=['IMDB_Rating'])

# Filtrer les films avec des évaluations supérieures à un certain seuil (par exemple, > 7)
df = df[df['IMDB_Rating'] > 7]


In [ ]:
# Encodage des genres en one-hot
genre_dummies = df['Genre'].str.get_dummies(', ')  # Si les genres sont séparés par des virgules
df = pd.concat([df, genre_dummies], axis=1)

# Encodage du certificat de film
df['Certificate'] = df['Certificate'].astype('category').cat.codes


In [ ]:
# Nettoyer la colonne 'Gross' en supprimant les caractères non numériques
df['Gross'] = df['Gross'].apply(lambda x: re.sub(r'[^\d.]', '', str(x)))

# Convertir la colonne 'Gross' en numérique
df['Gross'] = pd.to_numeric(df['Gross'], errors='coerce')

# Remplir les valeurs manquantes dans 'Gross' par la moyenne
df['Gross'] = df['Gross'].fillna(df['Gross'].mean())


In [ ]:
# Convertir la colonne 'IMDB_Rating' en numérique et supprimer les lignes avec des valeurs manquantes
df['IMDB_Rating'] = pd.to_numeric(df['IMDB_Rating'], errors='coerce')
df = df.dropna(subset=['IMDB_Rating'])

# Encodage des genres en one-hot
genre_dummies = df['Genre'].str.get_dummies(', ')  # Si les genres sont séparés par des virgules
df = pd.concat([df, genre_dummies], axis=1)

# Encodage du certificat de film
df['Certificate'] = df['Certificate'].astype('category').cat.codes

# Supprimer les lignes avec des valeurs manquantes
df = df.dropna()

# Vérifier les premières lignes après le prétraitement
print(df.head())

               Series_Title Released_Year  Certificate  Runtime  \
0  The Shawshank Redemption          1994            2  142 min   
1             The Godfather          1972            2  175 min   
2           The Dark Knight          2008           15  152 min   
3    The Godfather: Part II          1974            2  202 min   
4              12 Angry Men          1957           13   96 min   

                  Genre  IMDB_Rating  \
0                 Drama          9.3   
1          Crime, Drama          9.2   
2  Action, Crime, Drama          9.0   
3          Crime, Drama          9.0   
4          Crime, Drama          9.0   

                                            Overview              Director  \
0  two imprisoned men bond number years finding s...        Frank Darabont   
1  organized crime dynastys aging patriarch trans...  Francis Ford Coppola   
2  menace known joker wreaks havoc chaos people g...     Christopher Nolan   
3  early life career vito corleone new york 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiate the TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')

# Fit and transform the 'Overview' column into a TF-IDF matrix
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Overview'])

# Convert the sparse matrix to a dense matrix and then to a DataFrame for easier inspection
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Check the shape and the first few words in the matrix
print(tfidf_df.shape)  # Should print the number of movies by number of features (words)
print(tfidf_df.head())


(1000, 5000)
   abandoned  abandons  abbey  abducted  abductions  abductor  aberration  \
0        0.0       0.0    0.0       0.0         0.0       0.0         0.0   
1        0.0       0.0    0.0       0.0         0.0       0.0         0.0   
2        0.0       0.0    0.0       0.0         0.0       0.0         0.0   
3        0.0       0.0    0.0       0.0         0.0       0.0         0.0   
4        0.0       0.0    0.0       0.0         0.0       0.0         0.0   

   abilities  ability  abkhazia  ...  zare  zealand  zealander  zero  zodiac  \
0        0.0  0.00000       0.0  ...   0.0      0.0        0.0   0.0     0.0   
1        0.0  0.00000       0.0  ...   0.0      0.0        0.0   0.0     0.0   
2        0.0  0.27671       0.0  ...   0.0      0.0        0.0   0.0     0.0   
3        0.0  0.00000       0.0  ...   0.0      0.0        0.0   0.0     0.0   
4        0.0  0.00000       0.0  ...   0.0      0.0        0.0   0.0     0.0   

   zombie  zombiefilled  zombies  zone  zor

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate the cosine similarity between all movie pairs
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Check the similarity between the first movie and the rest
print(cosine_sim[0])  # Similarity of the first movie with all other movies


[1.         0.         0.         0.         0.         0.04229647
 0.07116109 0.         0.         0.         0.         0.
 0.04876836 0.         0.         0.         0.         0.
 0.         0.         0.09285806 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.0321553
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.0357133  0.         0.
 0.         0.         0.1657847  0.         0.         0.03808434
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.03720303 0.         0.         0.         0.         0.
 0.         0.03598648 0.         0.         0.         0.
 0.         0.         0.        

In [ ]:






from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Supposons que 'tfidf_matrix' est la matrice résultante de la vectorisation des résumés (comme défini dans les étapes précédentes)

# Calculer la similarité cosinus entre tous les films
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Vérifier la forme de la matrice de similarité
print(cosine_sim.shape)  # La taille devrait être (nombre de films, nombre de films)

# Exemple : calculer la similarité entre le premier film et tous les autres
print(cosine_sim[0])  # Cela renverra un tableau des similarités du film 0 avec tous les autres films


(1000, 1000)
[1.         0.         0.         0.         0.         0.04229647
 0.07116109 0.         0.         0.         0.         0.
 0.04876836 0.         0.         0.         0.         0.
 0.         0.         0.09285806 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.0321553
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.0357133  0.         0.
 0.         0.         0.1657847  0.         0.         0.03808434
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.03720303 0.         0.         0.         0.         0.
 0.         0.03598648 0.         0.         0.         0.
 0.         0.      

In [ ]:
def get_recommendations(movie_index, cosine_sim, top_n=5):
    # Récupérer les similarités pour un film donné (movie_index)
    similarity_scores = list(enumerate(cosine_sim[movie_index]))

    # Trier les films par similarité décroissante (ignorer le film lui-même)
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    # Obtenir les indices des N films les plus similaires
    movie_indices = [i[0] for i in similarity_scores[1:top_n+1]]  # Exclure le film lui-même

    return movie_indices

# Exemple : recommander les 5 films les plus similaires au film d'index 0
recommended_movies = get_recommendations(0, cosine_sim, top_n=5)

# Afficher les titres des films recommandés
for idx in recommended_movies:
    print(df['Series_Title'].iloc[idx])


Dev.D
Oldeuboi
The Great Escape
The Breakfast Club
Fa yeung nin wah


In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

#  df est la DataFrame contenant les films et leurs résumés (Overview)

# Étape 1 : Vectorisation des résumés (Overview)
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Overview'])

# Étape 2 : Calcul de la similarité cosinus entre tous les films
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Étape 3 : Fonction de recommandation basée sur un film donné
def recommend_movie(title, cosine_sim=cosine_sim, top_n=5):
    # Récupérer l'index du film basé sur le titre
    idx = df[df['Series_Title'] == title].index[0]

    # Récupérer les scores de similarité pour ce film
    similarity_scores = list(enumerate(cosine_sim[idx]))

    # Trier les films par similarité décroissante
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    # Sélectionner les indices des films les plus similaires (exclure le film lui-même)
    movie_indices = [i[0] for i in similarity_scores[1:top_n+1]]

    # Retourner les titres des films recommandés
    recommended_movies = df['Series_Title'].iloc[movie_indices].values
    return recommended_movies

# Exemple : recommander les 5 films les plus similaires à "The Shawshank Redemption"
recommended_movies = recommend_movie("The Shawshank Redemption", cosine_sim, top_n=5)

# Afficher les films recommandés
for movie in recommended_movies:
    print(movie)


Dev.D
Oldeuboi
The Great Escape
Fa yeung nin wah
Koe no katachi


In [ ]:
# Fonction fictive pour générer des films recommandés
def recommend_movie(movie_title, top_n=5):
    # Cette fonction simule une recommandation basée sur la similarité.
    # Remplacer cette logique par celle basée sur la similarité calculée plus tôt.
    # Exemple de films recommandés (c'est juste un exemple, tu peux appliquer ta logique ici)
    recommendations = {
        "The Shawshank Redemption": ["The Green Mile", "The Godfather", "Schindler's List"],
        "The Godfather": ["The Godfather: Part II", "Scarface", "Goodfellas"],
        "The Dark Knight": ["Inception", "The Prestige", "Joker"],
    }
    return recommendations.get(movie_title, [])[:top_n]

# Jeu de données de test avec des films pertinents pour chaque film testé
test_data = {
    "The Shawshank Redemption": ["The Green Mile", "The Godfather", "Schindler's List"],
    "The Godfather": ["The Godfather: Part II", "Scarface", "Goodfellas"],
    "The Dark Knight": ["Inception", "The Prestige", "Joker"],
}

# Fonction d'évaluation pour calculer la précision, le rappel et le score F1
def evaluate_recommendations(model, test_data, top_n=5):
    true_relevant = 0
    total_recommended = 0
    total_relevant = 0

    # On parcourt les films de test et leurs films pertinents
    for movie, relevant_movies in test_data.items():
        recommended_movies = model(movie, top_n=top_n)  # Obtenir les films recommandés
        total_relevant += len(relevant_movies)  # Nombre total de films pertinents

        # Vérification des films recommandés par rapport aux films pertinents
        for recommended_movie in recommended_movies:
            if recommended_movie in relevant_movies:
                true_relevant += 1

        total_recommended += top_n  # Nombre total de films recommandés

    # Calcul de la précision et du rappel
    precision = true_relevant / total_recommended if total_recommended > 0 else 0
    recall = true_relevant / total_relevant if total_relevant > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if precision + recall != 0 else 0

    return precision, recall, f1

# Exemple d'utilisation avec le jeu de données de test
precision, recall, f1 = evaluate_recommendations(recommend_movie, test_data)
print(f'Précision: {precision:.4f}, Rappel: {recall:.4f}, F1-score: {f1:.4f}')


Précision: 0.6000, Rappel: 1.0000, F1-score: 0.7500


In [ ]:
from google.colab import files
files.download('/content/imdb_movies_cleaned.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>